# urllib

## Preliminaries

Check that we are running the expected version of Python (should be something like 3.5.3).

In [2]:
import sys
sys.version

'3.6.2 (default, Jul 20 2017, 03:52:27) \n[GCC 7.1.1 20170630]'

Load some functions from `urllib`.

In [3]:
from urllib.parse import urlencode, urljoin
from urllib.request import urlopen, urlretrieve, HTTPError, URLError
from urllib.robotparser import RobotFileParser

Other packages.

In [4]:
import os

## Working with URLs

### URL Encoding

Some characters (like commas and spaces) need to be specially encoded when they are embedded in URLs. You can try this out [here](https://www.urlencoder.org/).

The `urlencode()` function accepts a dictionary then encodes and concatenates each of its elements.

In [5]:
params = {
    'q': 'web scraping',
    'ie': 'UTF-8'
}
urlencode(params)

'q=web+scraping&ie=UTF-8'

### Builing a URL

With `urljoin()` you can compose an absolute URL and a relative URL.

In [6]:
urljoin('https://www.google.com', '/search')

'https://www.google.com/search'

Now joining and encoding to form the complete URL for a Google query.

In [7]:
urljoin('https://www.google.com', '/search')+'?'+urlencode(params)

'https://www.google.com/search?q=web+scraping&ie=UTF-8'

## Retrieving a Web Page

URLs on the [Private Property](https://www.privateproperty.co.za/) yield a hierarchy of information. Consider the following:

`https://www.privateproperty.co.za/for-sale/kwazulu-natal/drakensberg/richmond-and-surrounds/byrne-valley/T479322`

It tells us that the property is

- "for sale" (as opposed to "to rent");
- in the province of KwaZulu-Natal;
- in the Drakensberg area;
- in the vicinity of Richmond;
- more specifically, the Byrne Valley; and
- has a *unique* ID of T479322.

It's the last bit that's really useful because it allows us to navigate directly to this property from the Home Page.

For brevity we'll use [bitly](https://bitly.com/) to generate a short URL.

In [18]:
URL = "http://bit.ly/2eL160q"

Retrieve the HTTP response.

In [9]:
html = urlopen(URL)

In [10]:
html

Take a look at the contents.

In [11]:
html.read()

b'\r\n<!DOCTYPE html>\r\n<html>\r\n<head>\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n    <meta property="fb:app_id" content="171012896285253" />\r\n\r\n    \r\n    <meta name="msapplication-config" content="/browserconfig.xml">\r\n\r\n    \r\n        <meta name="description" content="1.3 ha farm in Byrne Valley, A historic home in Byrne Valley.\nA chance to own one of the first dwellings ever built in Byrne Vall" />\r\n    \r\n\r\n<meta property="og:site_name" content="Private Property" />\r\n<meta property="og:locale" content="en_us" />\r\n<meta property="og:title" content="1.3 ha farm for sale in Byrne Valley | T479322 | Private Property" />\r\n        <meta property="og:image" content="https://prppublicstore.blob.core.windows.net/live-za-images/property/552/33/2366552/images/property-2366552-66071495_e.jpg" />\r\n        <meta property="og:image:width" content="600"/>\r\n        <meta property="og:image:height" content="450"/>\r\n<meta property="og:type" content

## More Robustly Retrieving the Page

Things can go wrong when you try to retrieve a web page. We have all had this experience in a browser. The same thing can happen in code.

Here are some possibilities:

- 404 error (page not found);
- 500 error (server error); or
- DNS failure or server not found.

Write code to cater for these cases.

In [19]:
# Try these URLs:
#
#URL = "https://www.privateproperty.co.za/for-sale/kwazulu-natal/durban/"
#URL = "http://www.totally-bogus-server.com/"

try:
	html = urlopen(URL)
except HTTPError as e:
	print(e)
    #
    # [Code to handle this error.]
except URLError as e:
	print(e)
    #
    # [Code to handle this error.]
else:
    print("Sucessfully retrieved the HTML for %s." % URL)
    #
    # This branch is not really necessary.

Sucessfully retrieved the HTML for http://bit.ly/2eL160q.


We've been using `urlopen()` which returns the contents of a URL as a string. What if we want to actually store a local copy of the URL? This can be handy for offline processing or if we want to archive a version of the page.

## Retrieving to a File

To download a URL to a local file use `urlretrieve()`.

In [20]:
filepath, http_message = urlretrieve(URL)

The return value is a tuple containing

- the path of the file in which the page is stored and
- a `HTTPMessage` object.

By default the file is downloaded to a temporary location.

In [21]:
filepath

'/tmp/tmpjss955er'

Check on the resulting file.

In [22]:
os.stat(filepath)

os.stat_result(st_mode=33152, st_ino=17, st_dev=38, st_nlink=1, st_uid=1000, st_gid=1000, st_size=87604, st_atime=1507121866, st_mtime=1507121866, st_ctime=1507121866)

The `HTTPMessage` object contains the response headers and behaves like a dictionary.

In [23]:
http_message.as_string()

'Cache-Control: private\nContent-Type: text/html; charset=utf-8\nServer: Microsoft-IIS/10.0\nX-AspNetMvc-Version: 5.2\nX-AspNet-Version: 4.0.30319\nSet-Cookie: live-za.phoenix.dv=1Eh2iIp5hECJLwCVlLYAxg; expires=Sun, 04-Oct-2116 12:57:45 GMT; path=/\nSet-Cookie: bondCalc_primeInterestRate=10.25; path=/\nSet-Cookie: bondCalc_txtInterestRate=10.25; path=/\nX-Powered-By: ASP.NET\nDate: Wed, 04 Oct 2017 12:57:45 GMT\nConnection: close\nContent-Length: 87604\n\n'

In [24]:
http_message.items()

[('Cache-Control', 'private'),
 ('Content-Type', 'text/html; charset=utf-8'),
 ('Server', 'Microsoft-IIS/10.0'),
 ('X-AspNetMvc-Version', '5.2'),
 ('X-AspNet-Version', '4.0.30319'),
 ('Set-Cookie',
  'live-za.phoenix.dv=1Eh2iIp5hECJLwCVlLYAxg; expires=Sun, 04-Oct-2116 12:57:45 GMT; path=/'),
 ('Set-Cookie', 'bondCalc_primeInterestRate=10.25; path=/'),
 ('Set-Cookie', 'bondCalc_txtInterestRate=10.25; path=/'),
 ('X-Powered-By', 'ASP.NET'),
 ('Date', 'Wed, 04 Oct 2017 12:57:45 GMT'),
 ('Connection', 'close'),
 ('Content-Length', '87604')]

In [25]:
http_message['Date']

'Wed, 04 Oct 2017 12:57:45 GMT'


We can also save the file to a specified location.

In [26]:
urlretrieve(URL, filename = 'T479322.html')

('T479322.html', <http.client.HTTPMessage at 0x7fe990072668>)

## Parsing `robots.txt`

Create an object for parsing `robots.txt`.

In [27]:
robots = RobotFileParser()

Let's work with the `robots.txt` from [PrivateProperty](https://www.privateproperty.co.za/robots.txt).

In [28]:
robots.set_url('https://www.privateproperty.co.za/robots.txt')
robots.read()

The `can_fetch()` method accepts two arguments:

- a User Agent and
- a URL.

In [29]:
robots.can_fetch('*', 'https://www.privateproperty.co.za/Portal/Home')

True

This URL is not allowed for anybody...

In [30]:
robots.can_fetch('*', 'https://www.privateproperty.co.za/Login')

False

... except Google, who is welcome.

In [31]:
robots.can_fetch('Mediapartners-Google', 'https://www.privateproperty.co.za/Login')

True

## Cleaning Up

In [ ]:
os.remove('T479322.html')